#  Training a neural network with TensorFlow

In [13]:
# tf 2.4
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import constant, Variable
from tensorflow import GradientTape, multiply
from sklearn.model_selection import train_test_split

### Initialization in TensorFlow
- A good initialization can reduce the amount of time needed to find the global minimum.
- we will initialize weights and biases for a neural network that will be used to predict credit card default decisions. To build intuition, we will use the low-level, linear algebraic approach, rather than making use of convenience functions and high-level keras operations.


In [86]:
# Define the layer 1 weights
w1 = Variable(tf.random.normal([23, 7]))

# Initialize the layer 1 bias
b1 = Variable(tf.ones([7]))

# Define the layer 2 weights
w2 = Variable(tf.random.normal([7,1]))

# Define the layer 2 bias
b2 = Variable(0.0)

print(b2)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>


### Defining the model and loss function
- train a neural network to predict whether a credit card holder will default.
- Note that the predictions layer is defined as `sigma(layer1*w2+b2)`, where is the sigmoid activation, layer1 is a tensor of nodes for the first hidden dense layer, w2 is a tensor of weights, and b2 is the bias tensor. 
- The trainable variables are w1, b1, w2, and b2


In [77]:
dataset = pd.read_csv('data/uci_credit_card.csv')
borrower_features_org = np.array(dataset.drop(['ID','default.payment.next.month'], axis=1))
default_org = np.array(dataset['default.payment.next.month']).reshape(-1,1)
print(borrower_features_org.shape)
print(default_org.shape)

(30000, 23)
(30000, 1)


In [80]:
# Define the model
def model(w1, b1, w2, b2, features = borrower_features):
    # Apply relu activation functions to layer 1
#     print(features.shape, w1.shape)
    layer1 = tf.keras.activations.relu(tf.matmul(features, w1) + b1)
    # Apply dropout
    dropout = tf.keras.layers.Dropout(0.25)(layer1)
    return tf.keras.activations.sigmoid(tf.matmul(dropout, w2) + b2)

# Define the loss function
def loss_function(w1, b1, w2, b2, features = borrower_features, targets = default):
    predictions = model(w1, b1, w2, b2)
    # Pass targets and predictions to the cross entropy loss
    return tf.keras.losses.binary_crossentropy(targets, predictions)

##### One of the benefits of using tensorflow is that we have the option to customize models down to the linear algebraic-level

## Training neural networks with TensorFlow
- train the model and then evaluate its performance by predicting default outcomes in a test set
- The trainable variables are w1, b1, w2, and b2. 

In [79]:
borrower_features, test_features, default , test_targets = train_test_split(borrower_features_org, default_org, test_size=0.2)
print(borrower_features.shape, test_features.shape )

(24000, 23) (6000, 23)


In [81]:
# define optimizer
opt = tf.keras.optimizers.Adam()

# Train the model
for j in range(100):
    # Complete the optimizer
    opt.minimize(lambda: loss_function(w1, b1, w2, b2), 
                 var_list=[w1, b1, w2, b2])

# Make predictions with model
model_predictions = model(w1, b1, w2, b2, test_features)

# Construct the confusion matrix
confusion_matrix(test_targets, model_predictions)

InvalidArgumentError: cannot compute MatMul as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:MatMul]